# 수정중인 코드입니다
# 최종추정일 : 2023.12.12

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument('lang=ko_KR')
driver = webdriver.Chrome(options=options)

def menu(data):
    try:
        driver.get("https://map.naver.com/v5/search/" + data)
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe#searchIframe')))
        
        temp = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="_pcmap_list_scroll_container"]/ul')))
        buttons = temp.find_elements(By.TAG_NAME, 'a')
        
        if '이미지수' in buttons[0].text or buttons[0].text == '':
            buttons[1].send_keys(Keys.ENTER)
        else:
            buttons[0].send_keys(Keys.ENTER)

        WebDriverWait(driver, 10).until(EC.default_content())
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.CSS_SELECTOR, 'iframe#entryIframe')))

        div_elements = driver.find_elements(By.XPATH, '//*[@id="ibu_1"]')
        img_url = None
        for div_element in div_elements:
            img_url = div_element.get_attribute('style')[80:-3]
        return img_url
    except (NoSuchElementException, TimeoutException):
        print(f"Error processing {data}")
        return None

# CSV 파일 읽기
df = pd.read_csv('../asset/c4_general_cafeteria_chuncheon_opened_SS-HJ_addLatiLongti_menu-price_addedLabel_v2.csv', encoding='utf-8-sig')

# 새로운 컬럼 추가
df['ImageURL'] = None

# 데이터프레임을 순회하며 이미지 URL 업데이트
for index, row in df.iterrows():
    img_url = menu(row['cafeteria_name'])
    if img_url:
        df.at[index, 'ImageURL'] = img_url
    print(f"Processed: {row['cafeteria_name']} - {img_url}")

# 결과 저장
df.to_csv('imgurl.csv', index=False)

# 드라이버 종료
driver.quit()
